In [ ]:
num_features = 2
hidden_dim = 64
num_heads = 8
num_encoder_layers = 2
initial_layer = nn.Linear(num_features, hidden_dim)
max_routes_to_swap = 90
route_combination_head = nn.Linear(2 * hidden_dim, max_routes_to_swap + 1)
transformer = nn.TransformerEncoder(
    nn.TransformerEncoderLayer(hidden_dim, num_heads),
    num_encoder_layers
)
number_of_customers = 15

# Simple 15 nodes with 2 features
graph_p1 = torch.rand(number_of_customers, num_features)
graph_p2 = torch.rand(number_of_customers, num_features)


# for ease of testing just a simple layer
P1_emmbedding = initial_layer(graph_p1)
P2_emmbedding = initial_layer(graph_p2)
print("Embedding Shape: ", P1_emmbedding.shape)


## Parent 1
P1_number_of_routes = 6
P1_Cnode_to_route = torch.tensor([0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5])
P1_node_to_route_matrix = torch.zeros(number_of_customers, P1_number_of_routes)
P1_node_to_route_matrix[torch.arange(number_of_customers), P1_Cnode_to_route] = 1


## Parent 2
P2_number_of_routes = 4
P2_Cnode_to_route = torch.tensor([0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3])
P2_node_to_route_matrix = torch.zeros(number_of_customers, P2_number_of_routes)
P2_node_to_route_matrix[torch.arange(number_of_customers), P2_Cnode_to_route] = 1


P1_route_aggregating = torch.matmul(P1_node_to_route_matrix.t(), P1_emmbedding)
P2_route_aggregating = torch.matmul(P2_node_to_route_matrix.t(), P2_emmbedding)
print("Parent 1 Route features: ", P1_route_aggregating.shape)
print("Parent 2 Route features: ", P2_route_aggregating.shape)

a, b = torch.broadcast_tensors(P1_route_aggregating[:, None], P1_route_aggregating[None, :])
parent_route_combination_representations = torch.cat((a, b), -1)
print("parent_route_combo: ", parent_route_combination_representations.shape)

max_to_swap = min(P1_number_of_routes, P2_number_of_routes)
full_prediction = route_combination_head(parent_route_combination_representations)
print("Full prediction shape: ", full_prediction.shape)

# Actual size of the allowed matrix =  (P1_number_of_routes, P2_number_of_routes, max_to_swap)
logits = full_prediction[:P1_number_of_routes, :P2_number_of_routes, :max_to_swap + 1]
probs = torch.softmax(logits.flatten(), -1).view_as(logits)

print('Output: ', probs.shape, probs.sum())


highest_prob = probs.max()
SREX_param = torch.where(probs == highest_prob)
print(f"best SrexParams: P1x = {SREX_param[0].item()}, P2x = {SREX_param[1].item()}, NumRoutesMove = {SREX_param[2].item()}")


In [ ]:


num_features = 2
hidden_dim = 64
num_heads = 8
num_encoder_layers = 2
initial_layer = nn.Linear(num_features, hidden_dim)
max_routes_to_swap = 90
route_combination_head = nn.Linear(2 * hidden_dim, max_routes_to_swap + 1)
transformer = nn.TransformerEncoder(
    nn.TransformerEncoderLayer(hidden_dim, num_heads),
    num_encoder_layers
)

graph_batch_left_parent = torch.rand(15, 2)
graph_batch_right_parent = torch.rand(15, 2)


hidden_left = initial_layer(graph_batch_left_parent)
out = hidden_left
"""
# apply transformer per graph in batch
out = torch.zeros_like(hidden_left)
for batch_idx in range(3):
    # get nodes from batch
    hidden_batch = hidden_left[node_to_graph_left == batch_idx, :]
    out[node_to_graph_left == batch_idx, :] = transformer(hidden_batch)

# apply transformer per route in batch
out = torch.zeros_like(hidden_left)
for route_idx in range(6): # loop over routes
    # get nodes from batch
    hidden_batch = hidden_left[node_to_route_left == route_idx, :]
    out[node_to_route_left == route_idx, :] = transformer(hidden_batch)"""

# out = transformer(hidden, node_to_graph1) # use pytorch.geometric

# transformer(hidden).shape

graph_idx = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2])

batch_size = 3
# parents1 = 3 routes, 2 routes, 1 route
# parents = [
#   [[0, 1, 2], [3, 4], [5, 6]], # route 0, 1, 2
#   [[7, 8], [9, 10, 11]], # route 3, 4
#   [[12, 13, 14]] # route 5
# ]

node_to_route1 = torch.tensor([0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5])
route_to_graph1 = torch.tensor([0, 0, 0, 1, 1, 2])
node_to_graph1 = route_to_graph1[node_to_route1]



# parents2
route_to_graph = []

node_to_route = torch.tensor([0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])
node_to_route2 = torch.tensor([0, 0, 0, 0, 1, 1, 1, 1])

def indices_to_matrix(indices):
    matrix = torch.zeros(len(indices), indices.max())

number_of_customers = 15
number_of_routes = 6
node_to_route_matrix = torch.zeros(number_of_customers,number_of_routes)
node_to_route_matrix[torch.arange(number_of_customers), node_to_route1] = 1
node_to_route_matrix

# node_to_route_matrix2 = torch.zeros(8, 2)
# node_to_route_matrix2[torch.arange(8), node_to_route2] = 1
# node_to_route_matrix2

route_hidden_left = torch.matmul(node_to_route_matrix.t(), out)
route_hidden_right = torch.matmul(node_to_route_matrix.t(), out)

route_to_graph_left = route_to_graph1
route_to_graph_right = route_to_graph_left

for batch_idx in range(3):  # loop over batch
    routes_left = route_hidden_left[route_to_graph_left == batch_idx, :]
    routes_right = route_hidden_right[route_to_graph_right == batch_idx, :]
    print("Couple: ", batch_idx,"Parent1_routes: ", routes_left[:, None].shape, "Parent2_routes: ",routes_right[None, :].shape)

    a, b = torch.broadcast_tensors(routes_left[:, None], routes_right[None, :])
    parent_route_combination_representations = torch.cat((a, b), -1)
    print("parent_route_combo: ", parent_route_combination_representations.shape)

    max_to_swap = min(routes_left.shape[0], routes_right.shape[0])
    full_prediction = route_combination_head(parent_route_combination_representations)
    print("Full prediction shape: ", full_prediction.shape)
    logits = full_prediction[:, :, :max_to_swap + 1]

    # print(result.shape)
    probs = torch.softmax(logits.flatten(), -1).view_as(logits)

    print('Output: ', probs.shape, probs.sum())
    print("")
    training_label = torch.rand(*probs.shape)  # performance score for option
    training_probs = torch.softmax(logits.flatten(), -1).view_as(logits)

    #loss = torch.cross_entropy(probs.flatten(), training_probs.flatten())
    # or something like
    # loss = torch.cross_entropy_with_logits(logits.flatten(), torch.softmax(training_labels.flatten()))

    # result[i, j, k] # unnormalized probability for starting at route i from
    # parent left and route j from parent right swapping k routes (may be 0)

    # print(a.shape, b.shape)
    # print(torch.cat((a, b), -1).shape)
    # route_combination_head